In [442]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.select import Select
from webdriver_manager.chrome import ChromeDriverManager
chromeOptions = webdriver.ChromeOptions()
import pandas as pd
import numpy as np
import math

In [ ]:
driver = webdriver.Chrome(executable_path=ChromeDriverManager().install(), options = chromeOptions)
driver.get('https://www.dipres.gob.cl/597/w3-multipropertyvalues-15168-35324.html#proyecto_programa')
elems = driver.find_elements("xpath", "//a[@href]")
links = []
for elem in elems:
    links.append(elem.get_attribute("href"))

In [ ]:
links = pd.DataFrame(links)
xlss = links[links[0].str.contains('xls')]
xlss[1] = xlss[0].str.extract('([0-9][0-9][0-9][0-9][0-9][0-9])')
xlss[1] = xlss[1].astype('int')
xlss = xlss[(xlss[1]>=289355)&(xlss[1]<=289717)]
xlss

In [ ]:
import time

In [ ]:
i = 0
for url in xlss[0]:
    print(url)
    file1 = pd.read_excel(url)
    file1.to_excel('xls/' +str(i)+ '.xls')
    i+=1
    time.sleep(2)

In [ ]:
import os
import pandas as pd
path = os.getcwd() + '/xls/'
files = os.listdir(path)


In [ ]:
lista = []
for f in files:
    data = pd.read_excel('xls/' + f)
    forma = list(data.shape)
    forma.append(f)
    lista.append(forma)


In [494]:
df = pd.DataFrame(lista)
df[df[1]==13]

,0,1,2
91,49,13,9.xls
155,49,13,152.xls
220,25,13,7.xls
230,29,13,6.xls
339,35,13,136.xls


In [495]:
dftotal = pd.DataFrame(columns=['subtitulo', 'item', 'asignacion', 'n_asignacion', 'glosa',
       'moneda_nacional_ms', 'moneda_internacional_musd', 'n_subtitulo',
       'n_item', 'partida', 'n_partida', 'capitulo', 'n_capitulo', 'programa',
       'n_programa'])
       
for row in df.iterrows():
    print(row[1][1], row[1][2])
    if row[1][1] == 17:
        dft = pd.read_excel('xls/'+ row[1][2])
        n_partida = dft['Unnamed: 1'][3]
        n_capitulo = dft['Unnamed: 1'][6]
        n_programa = dft['Unnamed: 1'][9]
        partida = dft['Unnamed: 15'][1]
        capitulo = dft['Unnamed: 15'][4]
        programa = dft['Unnamed: 15'][7]

        # Seleccionamos solo las filas con datos presupuestarios
        last = dft.index[dft['Unnamed: 1'] == 'GLOSAS:'].to_list()[0]
        dft2 = dft.iloc[10:last]
        dft2 = dft2[['Unnamed: 1','Unnamed: 3', 'Unnamed: 4', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9']]
        dft2.dropna(how='all', inplace=True)

        # eliminamos las filas de agregación
        agregaciones = dft2.index[(dft2['Unnamed: 1'].isna())&(dft2['Unnamed: 3'].isna())&(dft2['Unnamed: 4'].isna())].to_list()
        dft2 = dft2.drop(agregaciones)
        dft2.reset_index(drop=True, inplace=True)

        #subtítulos
        dft2['n_subtitulo'] = '' #nombre subtitulo
        borrar = []
        temp = dft2[dft2['Unnamed: 1'].notna()]
        for i, subt in temp['Unnamed: 1'].iteritems():
                if type(subt) is str:
                        if i == len(dft2['Unnamed: 1'])-1:
                                if (type(dft2.at[i, 'Unnamed: 3']) is float) or (type(dft2.at[i, 'Unnamed: 3']) is np.float64):
                                        dft2.at[i, 'Unnamed: 3']=0
                                        dft2.at[i, 'Unnamed: 4']=0
                                        dft2.at[i, 'n_subtitulo'] = dft2.at[i, 'Unnamed: 6']
                                else:
                                        dft2.at[i, 'n_subtitulo'] = dft2.at[i, 'Unnamed: 6']
                        else: 
                                if type(dft2.at[i+1, 'Unnamed: 3']) is str:
                                        borrar.append(i)
                                        j=i+1
                                        while type(dft2.at[j, 'Unnamed: 1']) is float:
                                                dft2.at[j, 'Unnamed: 1'] = subt
                                                dft2.at[j, 'n_subtitulo'] = dft2.at[i, 'Unnamed: 6']
                                                j+=1
                                                if j >= (len(dft2['Unnamed: 1'])):
                                                        break
                                else:
                                        dft2.at[i, 'Unnamed: 3']=0
                                        dft2.at[i, 'Unnamed: 4']=0
                                        dft2.at[i, 'n_subtitulo'] = dft2.at[i, 'Unnamed: 6']
        dft2 = dft2[dft2['n_subtitulo']!='']
        dft2.reset_index(drop=True, inplace=True)

        #items
        dft2['n_item'] = '' #nombre item
        borrar = []
        temp = dft2[(dft2['Unnamed: 3'].notna())&(dft2['Unnamed: 3']!=0)]
        for i, subt in temp['Unnamed: 3'].iteritems():
                if (type(subt) is str):
                        if i == len(dft2['Unnamed: 3'])-1:
                                if (type(dft2.at[i, 'Unnamed: 4']) is float) or (type(dft2.at[i, 'Unnamed: 4']) is np.float64):
                                        dft2.at[i, 'Unnamed: 4']=0
                                        dft2.at[i, 'n_item'] = dft2.at[i, 'Unnamed: 6']
                                else:
                                        dft2.at[i, 'n_item'] = dft2.at[i, 'Unnamed: 6']
                        else:        
                                if (not math.isnan(dft2.at[i+1, 'Unnamed: 4'])) and (dft2.at[i+1, 'Unnamed: 4']!=0):
                                        borrar.append(i)
                                        j=i+1
                                        while (type(dft2.at[j, 'Unnamed: 3']) is float) or (type(dft2.at[j, 'Unnamed: 3']) is np.float64):
                                                dft2.at[j, 'Unnamed: 3'] = subt
                                                dft2.at[j, 'n_item'] = dft2.at[i, 'Unnamed: 6']
                                                j+=1
                                                if j >= (len(dft2['Unnamed: 3'])):
                                                        break
                                else:
                                        dft2.at[i, 'Unnamed: 4']=0
                                        dft2.at[i, 'n_item'] = dft2.at[i, 'Unnamed: 6']

        dft2 = dft2[~dft2['Unnamed: 4'].isna()]
        dft2 = dft2.iloc[1:]

        dft2.rename(columns={'Unnamed: 1': 'subtitulo', 'Unnamed: 3':'item', 'Unnamed: 4':'asignacion', 'Unnamed: 6':'n_asignacion',\
        'Unnamed: 7':'glosa', 'Unnamed: 8': 'moneda_nacional_ms', 'Unnamed: 9':'moneda_internacional_musd'}, inplace=True)
        dft2.loc[dft2['n_item']=='', 'n_item'] = '-'
        dft2.loc[dft2['asignacion']==0, 'n_asignacion'] = '-'
        dft2['partida'] = partida
        dft2['n_partida'] = n_partida
        dft2['capitulo'] = capitulo
        dft2['n_capitulo'] = n_capitulo
        dft2['programa'] = programa
        dft2['n_programa'] = n_programa

        dftotal = dftotal.append(dft2)
        
    if row[1][1] == 15:
        dft = pd.read_excel('xls/'+ row[1][2])
        n_partida = dft['Unnamed: 1'][3]
        n_capitulo = dft['Unnamed: 1'][6]
        n_programa = dft['Unnamed: 1'][9]
        partida = dft['Unnamed: 13'][1]
        capitulo = dft['Unnamed: 13'][4]
        programa = dft['Unnamed: 13'][7]

        # Seleccionamos solo las filas con datos presupuestarios
        if dft.index[dft['Unnamed: 1'] == 'GLOSAS:'].to_list() == []:
                last = len(dft)
        else:
                last = dft.index[dft['Unnamed: 1'] == 'GLOSAS:'].to_list()[0]
        dft2 = dft.iloc[10:last]
        dft2 = dft2[['Unnamed: 1','Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8']]
        dft2.dropna(how='all', inplace=True)

        # eliminamos las filas de agregación
        agregaciones = dft2.index[(dft2['Unnamed: 1'].isna())&(dft2['Unnamed: 3'].isna())&(dft2['Unnamed: 4'].isna())].to_list()
        dft2 = dft2.drop(agregaciones)
        dft2.reset_index(drop=True, inplace=True)
        
        #subtítulos
        dft2['n_subtitulo'] = '' #nombre subtitulo
        borrar = []
        temp = dft2[dft2['Unnamed: 1'].notna()]
        for i, subt in temp['Unnamed: 1'].iteritems():
                if type(subt) is str:
                        if i == len(dft2['Unnamed: 1'])-1:
                                if (type(dft2.at[i, 'Unnamed: 3']) is float) or (type(dft2.at[i, 'Unnamed: 3']) is np.float64):
                                        dft2.at[i, 'Unnamed: 3']=0
                                        dft2.at[i, 'Unnamed: 4']=0
                                        dft2.at[i, 'n_subtitulo'] = dft2.at[i, 'Unnamed: 5']
                                else:
                                        dft2.at[i, 'n_subtitulo'] = dft2.at[i, 'Unnamed: 5']
                        else: 
                                if type(dft2.at[i+1, 'Unnamed: 3']) is str:
                                        borrar.append(i)
                                        j=i+1
                                        while type(dft2.at[j, 'Unnamed: 1']) is float:
                                                dft2.at[j, 'Unnamed: 1'] = subt
                                                dft2.at[j, 'n_subtitulo'] = dft2.at[i, 'Unnamed: 5']
                                                j+=1
                                                if j >= (len(dft2['Unnamed: 1'])):
                                                        break
                                else:
                                        dft2.at[i, 'Unnamed: 3']=0
                                        dft2.at[i, 'Unnamed: 4']=0
                                        dft2.at[i, 'n_subtitulo'] = dft2.at[i, 'Unnamed: 5']
        dft2 = dft2[dft2['n_subtitulo']!='']
        dft2.reset_index(drop=True, inplace=True)

        #items
        dft2['n_item'] = '' #nombre item
        borrar = []
        temp = dft2[(dft2['Unnamed: 3'].notna())&(dft2['Unnamed: 3']!=0)]
        for i, subt in temp['Unnamed: 3'].iteritems():
                if (type(subt) is str):
                        if i == len(dft2['Unnamed: 3'])-1:
                                if (type(dft2.at[i, 'Unnamed: 4']) is float) or (type(dft2.at[i, 'Unnamed: 4']) is np.float64):
                                        dft2.at[i, 'Unnamed: 4']=0
                                        dft2.at[i, 'n_item'] = dft2.at[i, 'Unnamed: 5']
                                else:
                                        dft2.at[i, 'n_item'] = dft2.at[i, 'Unnamed: 5']
                        else:        
                                if (not math.isnan(dft2.at[i+1, 'Unnamed: 4'])) and (dft2.at[i+1, 'Unnamed: 4']!=0):
                                        borrar.append(i)
                                        j=i+1
                                        while (type(dft2.at[j, 'Unnamed: 3']) is float) or (type(dft2.at[j, 'Unnamed: 3']) is np.float64):
                                                dft2.at[j, 'Unnamed: 3'] = subt
                                                dft2.at[j, 'n_item'] = dft2.at[i, 'Unnamed: 5']
                                                j+=1
                                                if j >= (len(dft2['Unnamed: 3'])):
                                                        break
                                else:
                                        dft2.at[i, 'Unnamed: 4']=0
                                        dft2.at[i, 'n_item'] = dft2.at[i, 'Unnamed: 5']
        
        dft2 = dft2[~dft2['Unnamed: 4'].isna()]
        dft2 = dft2.iloc[1:]
        dft2.rename(columns={'Unnamed: 1': 'subtitulo', 'Unnamed: 3':'item', 'Unnamed: 4':'asignacion', 'Unnamed: 5':'n_asignacion',\
        'Unnamed: 6':'glosa', 'Unnamed: 7': 'moneda_nacional_ms'}, inplace=True)
        dft2.loc[dft2['n_item']=='', 'n_item'] = '-'
        dft2.loc[dft2['asignacion']==0, 'n_asignacion'] = '-'
        dft2['partida'] = partida
        dft2['n_partida'] = n_partida
        dft2['capitulo'] = capitulo
        dft2['n_capitulo'] = n_capitulo
        dft2['programa'] = programa
        dft2['n_programa'] = n_programa

        dftotal = dftotal.append(dft2)




15 138.xls
15 110.xls
15 104.xls
15 11.xls
15 39.xls
15 312.xls
15 306.xls
15 266.xls
15 272.xls
15 299.xls
15 298.xls
15 273.xls
15 267.xls
15 307.xls
15 313.xls
15 38.xls
15 10.xls
15 105.xls
15 111.xls
15 139.xls
15 107.xls
15 113.xls
15 12.xls
15 339.xls
15 305.xls
15 311.xls
15 259.xls
15 271.xls
15 265.xls
15 264.xls
15 270.xls
15 258.xls
15 310.xls
15 304.xls
15 338.xls
15 13.xls
15 112.xls
15 106.xls
15 102.xls
15 116.xls
15 17.xls
15 300.xls
15 314.xls
15 328.xls
15 274.xls
15 260.xls
15 248.xls
15 249.xls
15 261.xls
15 275.xls
15 329.xls
15 315.xls
15 301.xls
15 16.xls
15 117.xls
15 103.xls
15 115.xls
15 101.xls
15 129.xls
15 28.xls
15 14.xls
15 317.xls
15 303.xls
15 263.xls
15 277.xls
15 288.xls
15 289.xls
15 276.xls
15 262.xls
15 302.xls
15 316.xls
15 15.xls
15 29.xls
15 128.xls
15 100.xls
15 114.xls
15 173.xls
15 99.xls
15 167.xls
15 72.xls
15 66.xls
15 198.xls
15 8.xls
15 359.xls
15 239.xls
15 205.xls
15 211.xls
15 210.xls
15 204.xls
15 238.xls
17 358.xls
13 9.xls
15 67.x

IndexError: list index out of range

In [501]:
dft.index[dft['Unnamed: 1'] == 'GLOSAS:'].to_list() == []

True

In [487]:
dft = pd.read_excel('xls/138.xls')
n_partida = dft['Unnamed: 1'][3]
n_capitulo = dft['Unnamed: 1'][6]
n_programa = dft['Unnamed: 1'][9]
partida = dft['Unnamed: 13'][1]
capitulo = dft['Unnamed: 13'][4]
programa = dft['Unnamed: 13'][7]

print(partida, n_partida)
print(capitulo, n_capitulo)
print(programa, n_programa)

10.0 MINISTERIO DE JUSTICIA Y DERECHOS HUMANOS
3.0 Servicio Médico Legal
1.0 Servicio Médico Legal (01)


In [488]:
# Seleccionamos solo las filas con datos presupuestarios
last = dft.index[dft['Unnamed: 1'] == 'GLOSAS:'].to_list()[0]
dft2 = dft.iloc[10:last]
dft2 = dft2[['Unnamed: 1','Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8']]
dft2.dropna(how='all', inplace=True)

# eliminamos las filas de agregación
agregaciones = dft2.index[(dft2['Unnamed: 1'].isna())&(dft2['Unnamed: 3'].isna())&(dft2['Unnamed: 4'].isna())].to_list()
dft2 = dft2.drop(agregaciones)
dft2.reset_index(drop=True, inplace=True)
dft2

,Unnamed: 1,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8
0,Sub- Título,Ítem Asig.,NaN,Denominaciones,Glosa N°,Moneda Nacional Miles de $,NaN
1,07,NaN,NaN,INGRESOS DE OPERACIÓN,NaN,2.220,NaN
2,NaN,02,NaN,Venta de Servicios,NaN,2.220,NaN
3,08,NaN,NaN,OTROS INGRESOS CORRIENTES,NaN,52.629,NaN
4,NaN,01,NaN,Recuperaciones y Reembolsos por Licencias Médicas,NaN,10,NaN
5,NaN,02,NaN,Multas y Sanciones Pecuniarias,NaN,1.135,NaN
6,NaN,99,NaN,Otros,NaN,51.484,NaN
7,09,NaN,NaN,APORTE FISCAL,NaN,51.985.281,NaN
8,NaN,01,NaN,Libre,NaN,51.985.281,NaN
9,12,NaN,NaN,RECUPERACIÓN DE PRÉSTAMOS,NaN,10,NaN


In [489]:
#subtítulos
dft2['n_subtitulo'] = '' #nombre subtitulo
borrar = []
temp = dft2[dft2['Unnamed: 1'].notna()]
for i, subt in temp['Unnamed: 1'].iteritems():
        if type(subt) is str:
                if i == len(dft2['Unnamed: 1'])-1:
                        if (type(dft2.at[i, 'Unnamed: 3']) is float) or (type(dft2.at[i, 'Unnamed: 3']) is np.float64):
                                dft2.at[i, 'Unnamed: 3']=0
                                dft2.at[i, 'Unnamed: 4']=0
                                dft2.at[i, 'n_subtitulo'] = dft2.at[i, 'Unnamed: 5']
                        else:
                                dft2.at[i, 'n_subtitulo'] = dft2.at[i, 'Unnamed: 5']
                else: 
                        if type(dft2.at[i+1, 'Unnamed: 3']) is str:
                                borrar.append(i)
                                j=i+1
                                while type(dft2.at[j, 'Unnamed: 1']) is float:
                                        dft2.at[j, 'Unnamed: 1'] = subt
                                        dft2.at[j, 'n_subtitulo'] = dft2.at[i, 'Unnamed: 5']
                                        j+=1
                                        if j >= (len(dft2['Unnamed: 1'])):
                                                break
                        else:
                                dft2.at[i, 'Unnamed: 3']=0
                                dft2.at[i, 'Unnamed: 4']=0
                                dft2.at[i, 'n_subtitulo'] = dft2.at[i, 'Unnamed: 5']
dft2 = dft2[dft2['n_subtitulo']!='']
dft2.reset_index(drop=True, inplace=True)
dft2

,Unnamed: 1,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,n_subtitulo
0,Sub- Título,0,0.0,Denominaciones,Glosa N°,Moneda Nacional Miles de $,NaN,Denominaciones
1,07,02,NaN,Venta de Servicios,NaN,2.220,NaN,INGRESOS DE OPERACIÓN
2,08,01,NaN,Recuperaciones y Reembolsos por Licencias Médicas,NaN,10,NaN,OTROS INGRESOS CORRIENTES
3,08,02,NaN,Multas y Sanciones Pecuniarias,NaN,1.135,NaN,OTROS INGRESOS CORRIENTES
4,08,99,NaN,Otros,NaN,51.484,NaN,OTROS INGRESOS CORRIENTES
5,09,01,NaN,Libre,NaN,51.985.281,NaN,APORTE FISCAL
6,12,10,NaN,Ingresos por Percibir,NaN,10,NaN,RECUPERACIÓN DE PRÉSTAMOS
7,15,0,0.0,SALDO INICIAL DE CAJA,NaN,10,NaN,SALDO INICIAL DE CAJA
8,21,0,0.0,GASTOS EN PERSONAL,02,37.226.489,NaN,GASTOS EN PERSONAL
9,22,0,0.0,BIENES Y SERVICIOS DE CONSUMO,03,14.722.385,NaN,BIENES Y SERVICIOS DE CONSUMO


In [490]:
#items
dft2['n_item'] = '' #nombre item
borrar = []
temp = dft2[(dft2['Unnamed: 3'].notna())&(dft2['Unnamed: 3']!=0)]
for i, subt in temp['Unnamed: 3'].iteritems():
        if (type(subt) is str):
                if i == len(dft2['Unnamed: 3'])-1:
                        if (type(dft2.at[i, 'Unnamed: 4']) is float) or (type(dft2.at[i, 'Unnamed: 4']) is np.float64):
                                dft2.at[i, 'Unnamed: 4']=0
                                dft2.at[i, 'n_item'] = dft2.at[i, 'Unnamed: 5']
                        else:
                                dft2.at[i, 'n_item'] = dft2.at[i, 'Unnamed: 5']
                else:        
                        if (not math.isnan(dft2.at[i+1, 'Unnamed: 4'])) and (dft2.at[i+1, 'Unnamed: 4']!=0):
                                borrar.append(i)
                                j=i+1
                                while (type(dft2.at[j, 'Unnamed: 3']) is float) or (type(dft2.at[j, 'Unnamed: 3']) is np.float64):
                                        dft2.at[j, 'Unnamed: 3'] = subt
                                        dft2.at[j, 'n_item'] = dft2.at[i, 'Unnamed: 5']
                                        j+=1
                                        if j >= (len(dft2['Unnamed: 3'])):
                                                break
                        else:
                                dft2.at[i, 'Unnamed: 4']=0
                                dft2.at[i, 'n_item'] = dft2.at[i, 'Unnamed: 5']


/Users/josemora/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [491]:
dft2 = dft2[~dft2['Unnamed: 4'].isna()]
dft2 = dft2.iloc[1:]


In [492]:
dft2.rename(columns={'Unnamed: 1': 'subtitulo', 'Unnamed: 3':'item', 'Unnamed: 4':'asignacion', 'Unnamed: 5':'n_asignacion',\
    'Unnamed: 6':'glosa', 'Unnamed: 7': 'moneda_nacional_ms'}, inplace=True)
dft2.loc[dft2['n_item']=='', 'n_item'] = '-'
dft2.loc[dft2['asignacion']==0, 'n_asignacion'] = '-'
dft2['partida'] = partida
dft2['n_partida'] = n_partida
dft2['capitulo'] = capitulo
dft2['n_capitulo'] = n_capitulo
dft2['programa'] = programa
dft2['n_programa'] = n_programa

In [493]:
dft2

,subtitulo,item,asignacion,n_asignacion,glosa,moneda_nacional_ms,Unnamed: 8,n_subtitulo,n_item,partida,n_partida,capitulo,n_capitulo,programa,n_programa
1,07,02,0.0,-,NaN,2.220,NaN,INGRESOS DE OPERACIÓN,Venta de Servicios,10.0,MINISTERIO DE JUSTICIA Y DERECHOS HUMANOS,3.0,Servicio Médico Legal,1.0,Servicio Médico Legal (01)
2,08,01,0.0,-,NaN,10,NaN,OTROS INGRESOS CORRIENTES,Recuperaciones y Reembolsos por Licencias Médicas,10.0,MINISTERIO DE JUSTICIA Y DERECHOS HUMANOS,3.0,Servicio Médico Legal,1.0,Servicio Médico Legal (01)
3,08,02,0.0,-,NaN,1.135,NaN,OTROS INGRESOS CORRIENTES,Multas y Sanciones Pecuniarias,10.0,MINISTERIO DE JUSTICIA Y DERECHOS HUMANOS,3.0,Servicio Médico Legal,1.0,Servicio Médico Legal (01)
4,08,99,0.0,-,NaN,51.484,NaN,OTROS INGRESOS CORRIENTES,Otros,10.0,MINISTERIO DE JUSTICIA Y DERECHOS HUMANOS,3.0,Servicio Médico Legal,1.0,Servicio Médico Legal (01)
5,09,01,0.0,-,NaN,51.985.281,NaN,APORTE FISCAL,Libre,10.0,MINISTERIO DE JUSTICIA Y DERECHOS HUMANOS,3.0,Servicio Médico Legal,1.0,Servicio Médico Legal (01)
6,12,10,0.0,-,NaN,10,NaN,RECUPERACIÓN DE PRÉSTAMOS,Ingresos por Percibir,10.0,MINISTERIO DE JUSTICIA Y DERECHOS HUMANOS,3.0,Servicio Médico Legal,1.0,Servicio Médico Legal (01)
7,15,0,0.0,-,NaN,10,NaN,SALDO INICIAL DE CAJA,-,10.0,MINISTERIO DE JUSTICIA Y DERECHOS HUMANOS,3.0,Servicio Médico Legal,1.0,Servicio Médico Legal (01)
8,21,0,0.0,-,02,37.226.489,NaN,GASTOS EN PERSONAL,-,10.0,MINISTERIO DE JUSTICIA Y DERECHOS HUMANOS,3.0,Servicio Médico Legal,1.0,Servicio Médico Legal (01)
9,22,0,0.0,-,03,14.722.385,NaN,BIENES Y SERVICIOS DE CONSUMO,-,10.0,MINISTERIO DE JUSTICIA Y DERECHOS HUMANOS,3.0,Servicio Médico Legal,1.0,Servicio Médico Legal (01)
10,23,03,0.0,-,NaN,10,NaN,PRESTACIONES DE SEGURIDAD SOCIAL,Prestaciones Sociales del Empleador,10.0,MINISTERIO DE JUSTICIA Y DERECHOS HUMANOS,3.0,Servicio Médico Legal,1.0,Servicio Médico Legal (01)


In [391]:
type(temp['Unnamed: 4'][3])

numpy.float64

In [403]:
not math.isnan(dft2.at[6, 'Unnamed: 4'])

True

In [430]:
dft2.keys()

Index(['subtitulo', 'item', 'asignacion', 'n_asignacion', 'glosa',
       'moneda_nacional_ms', 'moneda_internacional_musd', 'n_subtitulo',
       'n_item', 'partida', 'n_partida', 'capitulo', 'n_capitulo', 'programa',
       'n_programa'],
      dtype='object')